In [1]:
! pip install transformers torch --quiet


In [5]:
from huggingface_hub import login
login()

ImportError: The `notebook_login` function can only be used in a notebook (Jupyter or Colab) and you need the `ipywidgets` module: `pip install ipywidgets`.

In [16]:
from transformers import AutoTokenizer, AutoModelForCausalLM
from tqdm.auto import tqdm  # for progress bars
from huggingface_hub import snapshot_download
# First download the model files with progress bar
model_id = "HuggingFaceTB/SmolLM2-1.7B-Instruct"
local_files = snapshot_download(
    repo_id=model_id,
    local_dir="./my_local_model",
    local_dir_use_symlinks=False,
    tqdm_class=tqdm
)


Fetching 24 files:  33%|███▎      | 8/24 [00:01<00:01,  8.04it/s]Could not set the permissions on the file 'my_local_model/.cache/huggingface/download/runs/Oct31_06-24-59_ip-26-0-174-36/events.out.tfevents.1730363825.ip-26-0-174-36.3169719.1.b3d7723fd0715ce6dcbccf7bb2097f59490b0ac670f798f5378ef5abb7d1301d.incomplete'. Error: [Errno 2] No such file or directory: 'my_local_model/runs/tmp_01d9b7f7-05bb-4c52-9178-12906a102da7'.
Continuing without setting permissions.
Could not set the permissions on the file 'my_local_model/.cache/huggingface/download/runs/Oct31_06-24-59_ip-26-0-174-36/events.out.tfevents.1730356365.ip-26-0-174-36.3169719.0.e6bfce1916438dd2e6553aa0a62d418087b3ae04f8af75e714ad1f01b7663db6.incomplete'. Error: [Errno 2] No such file or directory: 'my_local_model/runs/tmp_cfc2d204-cfab-452b-a498-b4cf0b91323c'.
Continuing without setting permissions.
Fetching 24 files:  33%|███▎      | 8/24 [00:12<00:01,  8.04it/s]Error while downloading from https://cdn-lfs-us-1.hf.co/repos/e4

ChunkedEncodingError: ('Connection broken: IncompleteRead(511466261 bytes read, 2911311691 more expected)', IncompleteRead(511466261 bytes read, 2911311691 more expected))

In [18]:
# Enable progress bars for downloads
tokenizer = AutoTokenizer.from_pretrained(
    "./my_local_model",
    show_progress=True  # Shows download progress
)
inputs = tokenizer("Hello, how are you?", return_tensors="pt")
# print each individual token decoded
for i, token in enumerate(inputs.input_ids[0]):
    print(f"{i}: {tokenizer.decode(token, skip_special_tokens=True)}")

# Load model with progress bar
model = AutoModelForCausalLM.from_pretrained(
    "./my_local_model",
    show_progress=True,  # Shows download progress
)

# For generation, you can add a progress bar too

outputs = model.generate(
    **inputs,
    max_new_tokens=100,  # Adjust as needed
    num_beams=4,        # Adjust as needed
    use_cache=True,
    progress_bar=True   # Shows generation progress
)

print(tokenizer.decode(outputs[0], skip_special_tokens=True))

0: Hello
1: ,
2:  how
3:  are
4:  you
5: ?


OSError: Error no file named pytorch_model.bin, model.safetensors, tf_model.h5, model.ckpt.index or flax_model.msgpack found in directory ./my_local_model.

In [19]:
from transformers import AutoTokenizer, AutoModelForCausalLM
from tqdm.auto import tqdm  # for progress bars
from huggingface_hub import snapshot_download
# First download the model files with progress bar
model_id = "meta-llama/Llama-3.2-1B-Instruct"
local_files = snapshot_download(
    repo_id=model_id,
    local_dir="./my_local_llama",
    local_dir_use_symlinks=False,
    tqdm_class=tqdm
)


Fetching 13 files: 100%|██████████| 13/13 [13:22<00:00, 61.71s/it]


In [20]:
from transformers import AutoModelForCausalLM, AutoTokenizer

# Load the model and tokenizer from local directory
model_path = "./my_local_llama"
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForCausalLM.from_pretrained(model_path)

# Print model architecture
print(model)

# Get number of parameters
total_params = sum(p.numel() for p in model.parameters())
print(f"\nTotal parameters: {total_params:,}")

# You can also inspect specific layers
print("\nModel config:")
print(model.config)

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(128256, 2048)
    (layers): ModuleList(
      (0-15): 16 x LlamaDecoderLayer(
        (self_attn): LlamaSdpaAttention(
          (q_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (k_proj): Linear(in_features=2048, out_features=512, bias=False)
          (v_proj): Linear(in_features=2048, out_features=512, bias=False)
          (o_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=2048, out_features=8192, bias=False)
          (up_proj): Linear(in_features=2048, out_features=8192, bias=False)
          (down_proj): Linear(in_features=8192, out_features=2048, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((2048,), eps=1e-05)
        (post_attention_layernorm): LlamaRMSNorm((2048,), eps=1e-05)
      )
    )
    (norm):

In [22]:
!pip freeze | grep numpy

python(31802) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


numpy==1.26.4


In [23]:
# Prepare input text
text = "What is machine learning?"

# Tokenize input
inputs = tokenizer(text, return_tensors="pt")
print(inputs)

# Generate response
outputs = model.generate(
    inputs.input_ids,
    max_length=200,  # Maximum length of the output
    num_beams=4,     # Beam search for better quality
    temperature=0.7, # Control randomness (higher = more random)
    top_p=0.9,      # Nucleus sampling
    pad_token_id=tokenizer.pad_token_id,
    eos_token_id=tokenizer.eos_token_id,
)

# Decode and print the response
response = tokenizer.decode(outputs[0], skip_special_tokens=True)
print(response)

{'input_ids': tensor([[128000,   3923,    374,   5780,   6975,     30]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1]])}


ImportError: cannot import name 'NP_SUPPORTED_MODULES' from 'torch._dynamo.utils' (/Users/hugoromero/Documents/GitHub/ai-projects/.conda/lib/python3.12/site-packages/torch/_dynamo/utils.py)